In [1]:
# control analysis with BOLD signal variability
import pandas as pd
from nilearn import plotting, image, datasets
from nilearn.maskers import NiftiMapsMasker
import nibabel as nib

# load subList of subjects with PLV data
with open('subList.txt') as f:
    subjects = f.read().split('\n')
    
# remove subjects rejected from mriqc:
subs2rej = ["0019", "0034", "0039", "0132", "0138", "0180", "0229", "0397", "0419", "0437", "0587", "0351", "0388", "0454", "0485", "0173", "0207", "0036", "0223", "0187", "0237", "0273", "0287"] 
subjects = [subject for subject in subjects if subject not in subs2rej]

#subjects = subjects[0:2]

dim = 256
all_subjects_cols = ['subject'] + [f'parcel_{i+1}' for i in range(dim)]
all_subjects = pd.DataFrame(columns=all_subjects_cols)
for subject in subjects:

    if subject == "":
        continue
    
    # load BOLD signal variability (standard deviation of timeseries post-retroicor/confound regressors removed)
    data_path = "/mnt/fast_scratch/StomachBrain/data/fMRI_timeseries"
    STD_path = f"{data_path}/sub-{subject}/sub-{subject}_STDxVoxel_RP.nii.gz"
    STD_img = nib.load(STD_path)

    # load Dictionaries of Functional Modes (“DiFuMo”) atlas
    difumo = datasets.fetch_atlas_difumo(dimension=dim, resolution_mm=3)

    # Create a masker object for difumo - with mask
    masker = NiftiMapsMasker(maps_img=difumo.maps) 

    # Extract the parcellated regions
    STD_parcels = masker.fit_transform(STD_img)
    
    # Append the subject and STD_parcels to the DataFrame
    subject_data = [subject] + list(STD_parcels[0])
    all_subjects = all_subjects.append(pd.Series(subject_data, index=all_subjects_cols), ignore_index=True)

save_path = '/home/leah/Git/StomachBrain/StomachBrainCCA/data/'  
all_subjects.to_csv(save_path + f'controlSTD_difumo{dim}_mriqc.csv', index=False)